# Review of RDD operations

In [1]:
string_start = ["Ciao mi chiamo Davide e studio ingegneria al Politecnico di Torino. Quando non studio mi piace ascoltare la musica. In realtà ascolto la musica anche quando studio."]

input_file = '/user/s315054/text.txt'

in_RDD = sc.textFile(input_file)

In [2]:
flattened_RDD = in_RDD.flatMap(lambda l: l.split(' '))

flattened_RDD.collect()

['Ciao',
 'mi',
 'chiamo',
 'Davide',
 'e',
 'studio',
 'ingegneria',
 'al',
 'Politecnico',
 'di',
 'Torino.',
 'Quando',
 'non',
 'studio',
 'mi',
 'piace',
 'ascoltare',
 'la',
 'musica.',
 'In',
 'realtà',
 'ascolto',
 'la',
 'musica',
 'anche',
 'quando',
 'studio.']

In [3]:
sep_words_RDD = flattened_RDD.map(lambda l: l.lower().replace('.', ''))

sep_words_RDD.collect()

['ciao',
 'mi',
 'chiamo',
 'davide',
 'e',
 'studio',
 'ingegneria',
 'al',
 'politecnico',
 'di',
 'torino',
 'quando',
 'non',
 'studio',
 'mi',
 'piace',
 'ascoltare',
 'la',
 'musica',
 'in',
 'realtà',
 'ascolto',
 'la',
 'musica',
 'anche',
 'quando',
 'studio']

In [4]:
sep_words_RDD.countByValue()

defaultdict(int,
            {'ciao': 1,
             'mi': 2,
             'chiamo': 1,
             'davide': 1,
             'e': 1,
             'studio': 3,
             'ingegneria': 1,
             'al': 1,
             'politecnico': 1,
             'di': 1,
             'torino': 1,
             'quando': 2,
             'non': 1,
             'piace': 1,
             'ascoltare': 1,
             'la': 2,
             'musica': 2,
             'in': 1,
             'realtà': 1,
             'ascolto': 1,
             'anche': 1})

In [5]:
flattened_RDD.intersection(sep_words_RDD).collect()

['realtà',
 'chiamo',
 'e',
 'studio',
 'ingegneria',
 'di',
 'non',
 'la',
 'anche',
 'mi',
 'al',
 'piace',
 'ascolto',
 'ascoltare',
 'musica',
 'quando']

In [6]:
sep_words_RDD.first()

'ciao'

In [7]:
# This allows to use reverse order, but 2nd letter is disregarded...
sep_words_RDD.top(2, key=(lambda el: (-1*ord(el[0]))))

['al', 'ascoltare']

In [8]:
sep_words_RDD.takeOrdered(2)

['al', 'anche']

In [9]:
sep_words_RDD.reduce(lambda w1, w2: str(w1) + ' ' + str(w2))

'ciao mi chiamo davide e studio ingegneria al politecnico di torino quando non studio mi piace ascoltare la musica in realtà ascolto la musica anche quando studio'

In [10]:
sep_words_RDD.fold('', lambda w1, w2: w1 + ' ' + w2).replace('  ', ' ')

' ciao mi chiamo davide e studio ingegneria al politecnico di torino quando non studio mi piace ascoltare la musica in realtà ascolto la musica anche quando studio '

## Pair RDD

In [11]:
pair_RDD = sep_words_RDD.map(lambda w: (w, 1))

pair_RDD.collect()

[('ciao', 1),
 ('mi', 1),
 ('chiamo', 1),
 ('davide', 1),
 ('e', 1),
 ('studio', 1),
 ('ingegneria', 1),
 ('al', 1),
 ('politecnico', 1),
 ('di', 1),
 ('torino', 1),
 ('quando', 1),
 ('non', 1),
 ('studio', 1),
 ('mi', 1),
 ('piace', 1),
 ('ascoltare', 1),
 ('la', 1),
 ('musica', 1),
 ('in', 1),
 ('realtà', 1),
 ('ascolto', 1),
 ('la', 1),
 ('musica', 1),
 ('anche', 1),
 ('quando', 1),
 ('studio', 1)]

In [12]:
# Another approach: from initial file
pair_RDD_2 = in_RDD.flatMap(
    lambda l: map(lambda w: (w, 1), [el.lower().replace('.', '') for el in l.split(' ')]))

# Approach: use flat map to create an RDD of (`word`, 1)
# From the initial file, use flatMap() to define the rdd

# The function in the flat map performs a Python mapping between each line and a list
# of (`word`, 1) elements for each word in the line (in this case 1 line only)

pair_RDD_2.collect()

[('ciao', 1),
 ('mi', 1),
 ('chiamo', 1),
 ('davide', 1),
 ('e', 1),
 ('studio', 1),
 ('ingegneria', 1),
 ('al', 1),
 ('politecnico', 1),
 ('di', 1),
 ('torino', 1),
 ('quando', 1),
 ('non', 1),
 ('studio', 1),
 ('mi', 1),
 ('piace', 1),
 ('ascoltare', 1),
 ('la', 1),
 ('musica', 1),
 ('in', 1),
 ('realtà', 1),
 ('ascolto', 1),
 ('la', 1),
 ('musica', 1),
 ('anche', 1),
 ('quando', 1),
 ('studio', 1)]

In [13]:
# mapValues vs. flatMapValues:
pair_RDD_2.flatMapValues(lambda v: [v*10, v+2]).cogroup(pair_RDD).mapValues(lambda v: (list(v[0]), list(v[1]))).collect()

[('in', ([10, 3], [1])),
 ('realtà', ([10, 3], [1])),
 ('ciao', ([10, 3], [1])),
 ('chiamo', ([10, 3], [1])),
 ('davide', ([10, 3], [1])),
 ('e', ([10, 3], [1])),
 ('studio', ([10, 3, 10, 3, 10, 3], [1, 1, 1])),
 ('ingegneria', ([10, 3], [1])),
 ('politecnico', ([10, 3], [1])),
 ('di', ([10, 3], [1])),
 ('non', ([10, 3], [1])),
 ('la', ([10, 3, 10, 3], [1, 1])),
 ('anche', ([10, 3], [1])),
 ('mi', ([10, 3, 10, 3], [1, 1])),
 ('al', ([10, 3], [1])),
 ('torino', ([10, 3], [1])),
 ('piace', ([10, 3], [1])),
 ('ascolto', ([10, 3], [1])),
 ('quando', ([10, 3, 10, 3], [1, 1])),
 ('ascoltare', ([10, 3], [1])),
 ('musica', ([10, 3, 10, 3], [1, 1]))]

## Testing accumulators

In [8]:
myAcc = sc.accumulator(0)

def myCountLetter(word, letter):
    if len(word) > 3:
        if word.startswith(letter):
            myAcc.add(1)
        return True

long_words_RDD = sep_words_RDD.filter(lambda w: myCountLetter(w, 'a'))
out = long_words_RDD.collect()
print("Accumulator value: ", myAcc.value)
print(out)

Accumulator value:  3
['ciao', 'chiamo', 'davide', 'studio', 'ingegneria', 'politecnico', 'torino', 'quando', 'studio', 'piace', 'ascoltare', 'musica', 'realtà', 'ascolto', 'musica', 'anche', 'quando', 'studio']
